<a href="https://colab.research.google.com/github/arpitamangal/treatment-effect/blob/main/TreatmentEffect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# #if running on colab, uncomment this cell and upload file
# from google.colab import files
# files.upload()
# #(click on choose files and select files to upload)

## Treatment Effect
Estimate the treatment effects if a ‘different room is assigned’ as the treatment indicator and its effect on the room being ‘canceled’

In [ ]:
#display all the outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
#read dataset
import pandas as pd
df = pd.read_csv("hotel_cancellation.csv")
df.shape
df.head()

(102894, 8)

,Unnamed: 0,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,days_in_waiting_list,different_room_assigned,is_canceled
0,3,13,2015,27,1,0,False,False
1,4,14,2015,27,1,0,False,False
2,5,14,2015,27,1,0,False,False
3,7,9,2015,27,1,0,False,False
4,8,85,2015,27,1,0,False,True


In [ ]:
df['different_room_assigned'].value_counts()

False    91673
True     11221
Name: different_room_assigned, dtype: int64

In [ ]:
import numpy as np
df['TreatFlg'] = np.where(df['different_room_assigned'].astype(str).str.strip()=="True",1,0).astype(int)
df['TreatFlg'].value_counts()

0    91673
1    11221
Name: TreatFlg, dtype: int64

In [ ]:
df['is_canceled'].value_counts()

False    62733
True     40161
Name: is_canceled, dtype: int64

In [ ]:
df['CanceledFlg'] = np.where(df['is_canceled'].astype(str).str.strip()=="True",1,0).astype(int)
df['CanceledFlg'].value_counts()

0    62733
1    40161
Name: CanceledFlg, dtype: int64

In [ ]:
del df["Unnamed: 0"]
del df["different_room_assigned"]
del df["is_canceled"]

In [ ]:
df.describe()

,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,days_in_waiting_list,TreatFlg,CanceledFlg
count,102894.000000,102894.000000,102894.000000,102894.000000,102894.000000,102894.000000,102894.000000
mean,111.740092,2016.156977,27.339155,15.786771,2.619579,0.109054,0.390314
std,107.681013,0.706117,13.279990,8.794042,18.797440,0.311708,0.487823
min,0.000000,2015.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,26.000000,2016.000000,17.000000,8.000000,0.000000,0.000000,0.000000
50%,79.000000,2016.000000,28.000000,16.000000,0.000000,0.000000,0.000000
75%,169.000000,2017.000000,38.000000,24.000000,0.000000,0.000000,1.000000
max,629.000000,2017.000000,53.000000,31.000000,391.000000,1.000000,1.000000


In [ ]:
df.dtypes

lead_time                    int64
arrival_date_year            int64
arrival_date_week_number     int64
arrival_date_day_of_month    int64
days_in_waiting_list         int64
TreatFlg                     int64
CanceledFlg                  int64
dtype: object

In [ ]:
corr = df.corr()
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)
 .highlight_null(null_color='#f1f1f1')  # Color NaNs grey
 .set_precision(2))

<ipython-input-19-8c49964ea9b1>:5: FutureWarning: `null_color` is deprecated: use `color` instead
  (corr
<ipython-input-19-8c49964ea9b1>:5: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  (corr


,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,days_in_waiting_list,TreatFlg,CanceledFlg
lead_time,nan,nan,nan,nan,nan,nan,nan
arrival_date_year,0.04,nan,nan,nan,nan,nan,nan
arrival_date_week_number,0.12,-0.53,nan,nan,nan,nan,nan
arrival_date_day_of_month,-0.00,0.01,0.06,nan,nan,nan,nan
days_in_waiting_list,0.17,-0.06,0.02,0.02,nan,nan,nan
TreatFlg,-0.13,-0.10,0.02,-0.01,-0.00,nan,nan
CanceledFlg,0.28,0.00,0.00,-0.01,0.05,-0.24,nan


Implementing logistic regression to estimate the coefficient for the treatment parameter

In [ ]:
import statsmodels.api as sm
formula = 'CanceledFlg ~ lead_time + arrival_date_year + arrival_date_week_number + arrival_date_day_of_month + days_in_waiting_list + TreatFlg'
model = sm.Logit.from_formula(formula = formula, data=df)
result = model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.597308
         Iterations 14
                           Logit Regression Results                           
Dep. Variable:            CanceledFlg   No. Observations:               102894
Model:                          Logit   Df Residuals:                   102887
Method:                           MLE   Df Model:                            6
Date:                Mon, 17 Apr 2023   Pseudo R-squ.:                  0.1070
Time:                        23:42:13   Log-Likelihood:                -61459.
converged:                       True   LL-Null:                       -68825.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                   356.5464     23.369     15.257      0.000     310

In [ ]:
#find the exponential of the specified value
import math
print(math.exp(-2.55))

0.07808166600115317


#### The p-value for f-test of full model is ~0 hence the model is valid. The likelihood of the model using covariates is better than the null model. All the covariates are significant to predict response except Arrival date day of the month. It could be because it has high correlation with the arrival date week number, which is significant in predicting hotel cancellation. Arrival date day of the month does not expalin any additional variation in the response variable. P-value for treatment (TreatFlg) i.e. different room assigned is ~0 which is less than any significant alpha. Hence, we reject null pythosis. We conclude that the treatment effect of different_room_assigned on  hotel cancellation is statistically siginificant. They are negatively related, meaning if the customer is assigned a different room the probability of cancellation decreases by e^(-2.55) i.e 7.8%.  A negative estimate for the treatment effect indicates that customers for whom room was changes had a less odds of cancellation compared to other bookings.

## Double Logistic
Double logistic regression to measure the effect of ‘different room is assigned’ on the room being ‘canceled’ while controlling for confounders.

In [ ]:
import statsmodels.api as sm
formula = 'TreatFlg ~ lead_time + arrival_date_year + arrival_date_week_number + arrival_date_day_of_month + days_in_waiting_list'
model1 = sm.Logit.from_formula(formula = formula, data=df)
result1 = model1.fit()
print(result1.summary())

Optimization terminated successfully.
         Current function value: 0.329997
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:               TreatFlg   No. Observations:               102894
Model:                          Logit   Df Residuals:                   102888
Method:                           MLE   Df Model:                            5
Date:                Mon, 17 Apr 2023   Pseudo R-squ.:                 0.04219
Time:                        23:55:05   Log-Likelihood:                -33955.
converged:                       True   LL-Null:                       -35450.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                  1033.8356     35.098     29.456      0.000     965.

#### By regresssing the treatment (different room assigned) on the different covariates (lead time, arrival date year, week , month and days in waiting list) we observe a realtion between them. The p-value for the model is ~0 which is significant and hence the overall model is valid. We can conclude that there is relation between treatment and other independent variables. Since the relation is statistically significant, there will be confounding effects. We also use the predicted value of treatment from the model to control for the confounder effects.

In [ ]:
df.head()

,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,days_in_waiting_list,TreatFlg,CanceledFlg,predicted_treat
0,13,2015,27,1,0,0,0,0.241149
1,14,2015,27,1,0,0,0,0.240269
2,14,2015,27,1,0,0,0,0.240269
3,9,2015,27,1,0,0,0,0.244695
4,85,2015,27,1,0,0,1,0.183412


In [ ]:
df["predicted_treat"] = result1.predict(df)

In [ ]:
import statsmodels.api as sm
formula = 'CanceledFlg ~ lead_time + arrival_date_year + arrival_date_week_number + arrival_date_day_of_month + days_in_waiting_list + TreatFlg + predicted_treat '
model2 = sm.Logit.from_formula(formula = formula, data=df)
#alpha: penalty weight. If a scalar, the same penalty weight applies to all variables in the model. If a vector, it contains a penalty weight for each coefficient.
#alpha is set to zero to have un_penalized predicted_treat
result2 = model2.fit_regularized(method='l1', alpha = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0])
print(result2.summary())

/usr/local/lib/python3.9/dist-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.9/dist-packages/statsmodels/discrete/discrete_model.py:1872: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.5984470089250322
            Iterations: 28
            Function evaluations: 43
            Gradient evaluations: 28


/usr/local/lib/python3.9/dist-packages/statsmodels/base/l1_solvers_common.py:71: ConvergenceWarning: QC check did not pass for 7 out of 8 parameters
Try increasing solver accuracy or number of iterations, decreasing alpha, or switch solvers
  warnings.warn(message, ConvergenceWarning)
/usr/local/lib/python3.9/dist-packages/statsmodels/base/l1_solvers_common.py:144: ConvergenceWarning: Could not trim params automatically due to failed QC check. Trimming using trim_mode == 'size' will still work.
  warnings.warn(msg, ConvergenceWarning)


                           Logit Regression Results                           
Dep. Variable:            CanceledFlg   No. Observations:               102894
Model:                          Logit   Df Residuals:                   102886
Method:                           MLE   Df Model:                            7
Date:                Mon, 17 Apr 2023   Pseudo R-squ.:                  0.1053
Time:                        23:58:21   Log-Likelihood:                -61576.
converged:                       True   LL-Null:                       -68825.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                     0.0126     52.663      0.000      1.000    -103.205     103.231
lead_time                     0.0052      0.000     27.602      0.000       0.005     

In [ ]:
#find the exponential of the specified value
import math
print(math.exp(-2.5186))

0.08057232908620612


In the first stage, a logistic regression model is fit to predict the probability of the treatment (different room assigned) as a function of the risk factor and other covariates. In the second stage, a regularised logistic regression model is fit to predict the probability of the main outcome (hotel cancellation) as a function of the risk factor, other covariates, and the predicted probability of the main outcome from the first stage. Howerevr, in the second model we do not penalized the predicted_treat from the first model.

So, double logistic regression is not just fitting two separate logistic regression models, but rather fitting two logistic regression models in two stages, with the results from the first stage being used as a predictor in the second stage.

Controling for the confounder effects of the predictor variables we see that the  model to predict the hotel cancellation is vaild. the predicted value of different room assigned captures all the affect of confounding variables and the new estimate for the treatment is the actual treatment effect. The p-value for the treatment variable is ~0 which is less than any significant alpha, we reject null hypothesis that the treatment has no affect on response. We conclud that the effect of treatment on response is statistically significant. Different room assigned and hotel cancellation are negatively correlated. This implies that if a different room is assigned then the probability of cancellation reduces. It reduced by e^(-2.5186) which is 8% reduction. We also observe that the predicted_treat valriable is not significant implying that the actual effect of other covariates is lower than what we were observing without controlling for them. Additionally we see that After controlling for the confounder affects we observe that the actual treatment effect is slightly higher than what we observed withount controlling for the confounders.

## Bootstrapping
Estimate the standard error of the treatment effects.

In [ ]:
# Define the number of bootstrap resamples
n_resamples = 1000

# Initialize a matrix to store the treatment effect estimates
treat_effects = np.zeros((n_resamples, result2.params.shape[0] - 1))

# Use bootstrapping to estimate the standard error of the treatment effects
i = 0
while i < n_resamples:
    resample_index = np.random.choice(df.index, size = df.index.size, replace = True)
    resample = df.iloc[resample_index]
    formula = 'TreatFlg ~ lead_time + arrival_date_year + arrival_date_week_number + arrival_date_day_of_month + days_in_waiting_list'
    result1 = sm.Logit.from_formula(formula = formula, data=resample).fit()
    resample["predicted_treat"] = result1.predict(resample)
    formula = 'CanceledFlg ~ lead_time + arrival_date_year + arrival_date_week_number + arrival_date_day_of_month + days_in_waiting_list + TreatFlg + predicted_treat '
    result2 = sm.Logit.from_formula(formula = formula, data=resample).fit()
    treat_effects[i, :] = result2.params[:-1]
    i += 1
# Calculate the standard error of the treatment effects
treat_effects_se = treat_effects.std(axis=0)

# Print the standard errors of the treatment effect estimates
print('Standard errors of the treatment effects:')
print(treat_effects_se)

Streaming output truncated to the last 5000 lines.
         Current function value: 0.333049
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.592427
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.329244
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.592836
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.329035
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.592904
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.329314
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.592420
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.330485
         Iterations 8
Optimization terminated successfully.
         Current function value

In [ ]:
print(treat_effects_se)

[6.88055212e+01 1.97369672e-04 3.40816807e-02 1.49982116e-03
 1.66521253e-03 8.18284286e-04 4.66702138e-02]


#### In the earlier methods we just focused on the parameter estimate for the treatment effect. However, we did not take into account that uncertanity in our estimation. To account for that we need to look at the standard error. The standadrd errors observed in earlier model does not account for model selection and only apply independently. Hence, to estimate the standard error we rely on boot strapping methods. Bootsrap Mimic us going out to population again and collecting the data. If the original data is good representation of society than the method works really well. The method will be a reflection of collecting samples from society and we can calculate the statistic of interest which here is the standard error. The standard error in measurement of treatment is  0.045. The value is comparable to the standard error of estimate for the treatment using double logistic.